In [ ]:
import numpy as np
import pandas as pd
import matplotlib as ply
import os
from pathlib import Path

In [ ]:
path = Path('titanic')
df = pd.read_csv(path/'train.csv')
df

In [ ]:
# Find columns containing no values
df.isna().sum()

In [ ]:
# Calculate the mode for each column and fill 
modes = df.mode().iloc[0]
modes

In [ ]:
df['InCabin'] = df['Cabin'].isna()
df.InCabin = df.InCabin.astype(int)
df.InCabin.hist()

In [ ]:

# in-place replace NaN values with mode.

df.fillna(modes, inplace=True)
df.isna().sum()

In [ ]:
df.describe(include=[np.number])

In [ ]:
df.Age.hist()

In [ ]:
df.Fare.hist()

In [ ]:
df['LogFare'] = np.log1p(df.Fare)
df.LogFare.hist()

In [ ]:
df.SibSp.hist()

In [ ]:
pclasses = sorted(df.Pclass.unique())
pclasses

In [ ]:
df.describe(include=[object])

In [ ]:
df.head()

In [ ]:
df_columns = list(df.columns)
df_columns

In [ ]:
dummy_columns = ["Sex", "Pclass", "Embarked", "InCabin" ]
df = pd.get_dummies(df, columns=dummy_columns)
new_columns = list(df.columns)
new_columns

In [ ]:
added_columns = list(set(new_columns + dummy_columns) ^ set(df_columns))
added_columns

In [ ]:
from torch import tensor
import torch
Y = tensor(df.Survived)

In [ ]:
added_cols = added_columns

In [ ]:
indep_cols = ['Age', 'SibSp', 'Parch', 'LogFare'] + added_cols
t_indep = tensor(df[indep_cols].values, dtype=torch.float)
t_indep.shape

In [ ]:
# Rank of the tensor
len(t_indep.shape)

## Setting up a linear model


In [ ]:
torch.manual_seed(42)

n_coeff = t_indep.shape[1]
coeffs = torch.rand(n_coeff) - 0.5
coeffs.shape

In [ ]:
t_indep * coeffs

In [ ]:
vals, indices = t_indep.max(dim=0)
t_indep = t_indep / vals
t_indep

In [ ]:
t_indep.shape

In [ ]:
t_indep * coeffs

In [ ]:
preds = (t_indep*coeffs).sum(axis=1)

In [ ]:
preds[:10]

In [ ]:
loss = torch.abs(preds - Y).mean()
loss

In [ ]:
def calc_preds(coeffs, indeps): return (coeffs * indeps).sum(axis=1)
def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()

## Doing gradient descent step


In [ ]:
coeffs.requires_grad_() # in-place operation

In [ ]:
loss = calc_loss(coeffs, t_indep, Y)
loss

In [ ]:
loss.backward()

In [ ]:
coeffs.grad

In [ ]:
with torch.no_grad():
    coeffs.sub_(coeffs.grad * 0.1)
    print(calc_loss(coeffs, t_indep, Y))

## Training the linear model

In [ ]:
from fastai.data.transforms import RandomSplitter
trn_split, val_split = RandomSplitter(seed=42)(df)

In [ ]:
trn_indep, val_indep = t_indep[trn_split], t_indep[val_split]
trn_Y, val_Y = Y[trn_split], Y[val_split]
len(trn_indep), len(val_indep)

In [ ]:
len(trn_Y), len(val_Y)

In [ ]:
def init_coeffs(n_coeff): return (torch.rand(n_coeff)-0.5).requires_grad_()
def update_coeffs(coeffs, lr): coeffs.sub_(coeffs.grad * lr)

In [ ]:
def one_epoch(coeffs, lr):
    loss = calc_loss(coeffs, trn_indep, trn_Y)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, lr)
    loss.grad = None
    print(f'{loss:.3f}', end=';')

In [ ]:
def train_model(epoch=30, lr=0.01):
    torch.manual_seed(42)
    coeffs = init_coeffs(n_coeff)
    for i in range(epoch): one_epoch(coeffs, lr=lr)
    return coeffs


In [ ]:
coeffs = train_model(18, lr=0.02)

In [ ]:
def show_coeffs(): return dict(zip(indep_cols, coeffs.requires_grad_(False)))
show_coeffs()

## Measuring Accuracy

In [ ]:
preds = calc_preds(coeffs, val_indep)

In [ ]:
results = val_Y.bool() == ( preds > 0.5)
results[:10]

In [ ]:
results.float().mean()

In [ ]:
def acc(coeffs): return (val_Y.bool() == (calc_preds(coeffs, val_indep)>0.5)).float().mean()
acc(coeffs)

## Using sigmoid

In [ ]:
preds[:20]

In [ ]:
import sympy
sympy.plot("1/(1+exp(-x))", xlim=(-10,10))

In [ ]:
def calc_preds(coeffs, indeps): return torch.sigmoid((indeps * coeffs).sum(axis=1))

In [ ]:
coeffs = train_model(lr=2)

In [ ]:
acc(coeffs)

In [ ]:
show_coeffs()

## Submitting to Kaggle


In [ ]:
tst_df = pd.read_csv(path/'test.csv')

In [ ]:
tst_df['Fare'] = tst_df.Fare.fillna(0)
tst_df['LogFare'] = np.log1p(tst_df.Fare)

tst_df['InCabin'] = tst_df['Cabin'].isna()
tst_df.InCabin = tst_df.InCabin.astype(int)

In [ ]:
# in-place replace NaN values with mode.
tst_modes = tst_df.mode().iloc[0]
tst_df.fillna(tst_modes, inplace=True)
tst_df = pd.get_dummies(tst_df, columns=dummy_columns)

tst_indep = tensor(tst_df[indep_cols].values, dtype=torch.float)
tst_indep = tst_indep / vals


In [ ]:
tst_df['Survived'] = (calc_preds(tst_indep, coeffs) > 0.5).int()

In [ ]:
sub_df = tst_df[['PassengerId', 'Survived']]
sub_df.to_csv('sub.csv', index=False)

In [ ]:
!head sub.csv

## Using Matrix Product

In [ ]:
(val_indep * coeffs).sum(axis=1)

In [ ]:
val_indep@coeffs

In [ ]:
def calc_preds(coeffs, indeps): return torch.sigmoid(indeps@coeffs)

In [ ]:
def init_coeffs(n_coeff): return (torch.rand(n_coeff, 1)*0.1).requires_grad_()

In [ ]:
trn_Y = trn_Y[:, None]
val_Y = val_Y[:, None]

In [ ]:
coeffs = train_model(lr=2)

In [ ]:
acc(coeffs)

In [ ]:
coeffs

## A Neural Network

In [ ]:
def init_coeffs(n_coeffs, n_hidden=20):
    layer1 = (torch.rand(n_coeffs, n_hidden)-0.5)/n_hidden
    layer2 = torch.rand(n_hidden, 1)-0.3
    const = torch.rand(1)[0]
    return layer1.requires_grad_(), layer2.requires_grad_(), const.requires_grad_()

              

In [ ]:
import torch.nn.functional as F

def calc_preds(coeffs, indeps):
    l1, l2, const = coeffs
    res = F.relu(indeps@l1)
    res = res@l2 + const
    return torch.sigmoid(res)


In [ ]:
def update_coeffs(coeffs, lr):
    for layer in coeffs: layer.sub_(layer.grad * lr)

In [ ]:
coeffs = train_model(lr=1.4)

In [ ]:
acc(coeffs)

## Deep Learning

In [ ]:
def init_coeffs(coeffs):
    hiddens = [10,10]
    sizes = [n_coeff] + hiddens + [1]
    n = len(sizes)
    layers = [ (torch.rand(sizes[i], sizes[i+1])-0.5)/sizes[i+1]*3 for i in range(n-1) ]
    consts = [ (torch.rand(1)[0]-0.5)*0.1 for i in range(n-1) ]
    for l in layers+consts: l.requires_grad_()
    return layers, consts


In [ ]:
def calc_preds(coeffs, indeps):
    layers, consts = coeffs
    n = len(layers)
    res = indeps
    for i, l in enumerate(layers):
        res = res@l + consts[i]
        if i!=n-1: res = F.relu(res)
    return torch.sigmoid(res)


In [ ]:
def update_coeffs(coeffs, lr):
    layers, consts = coeffs
    for layer in layers+consts: layer.sub_(layer.grad * lr)

In [ ]:
coeffs = train_model(lr=1.2)

In [ ]:
acc(coeffs)